<a href="https://colab.research.google.com/github/Sormy23/DataScience_CoronaStatistics/blob/main/mini_projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid Status
This Data Science project by Sven Oberwalder and Yasin Sahin analyzes the development of the corona pandemic based on input data provided by Statistik Austria

The Datasets can be found here:
### [Dataset 1](https://data.statistik.gv.at/web/meta.jsp?dataset=OGD_covidggstatus2_GGSTATUS_2)

### [Dataset 2](https://data.statistik.gv.at/web/meta.jsp?dataset=OGD_covidggstatus_GGSTATUS_1)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

In [ ]:
#Use mount to google drive when working on google colab
drive.mount('/content/drive')

Mounted at /content/drive


# Initial Data Analysis and Import
Dataset 1 has following attributes:
* **C-BEZIMST-0** Politischer Bezirk (PolBez)
* **C-ALTGRIMST-0** 5-years age group (Altersgr)
* **C-GLIMST-0** Country of Birth (GebLand)
* **C-C11-0** Sex (Geschl)
* **C-IMST-0** COVID-19 vaccinated-recovered-status (GeimpftGenesen)
* **F-DATA** Number of records (Anz)

Dataset 2 has following attributes:
* **C-B00-0** Federal country (Bundesland)
* **C-BILIMST-0** Education (Bildung)
* **C-ALTGRIMST-0** 5-years age group (Altersgr)
* **C-ESIMST-0** Economic status (ErwerbStatus)
* **C-IMST-0** COVID-19 vaccinated-recovered-status (GeimpftGenesen)
* **F-DATA** Number of records (Anz)

In below code, the datasets need to be imported and the attributes must be renamed to more meaningful names. Since the data is separated by semi-colons (;), we need to specify that when importing the csv-files. Furthermore, a simple Std-Analysis has to be conducted, to ensure the progress.

In [ ]:
#import databases
dataset1 = pd.read_csv("/content/drive/MyDrive/dataset1.csv", sep=";")
dataset2 = pd.read_csv("/content/drive/MyDrive/dataset2.csv", sep=";")

#rename attributes
dataset1.rename(columns={"C-BEZIMST-0": "PolBez",
                         "C-ALTGRIMST-0": "Altersgr",
                         "C-GLIMST-0": "GebLand",
                         "C-C11-0": "Geschl",
                         "C-IMST-0": "GeimpftGenesen",
                         "F-DATA": "Anz"}, inplace=True)
dataset2.rename(columns={"C-B00-0": "Bundesland",
                         "C-BILIMST-0": "Bildung",
                         "C-ALTGRIMST-0": "Altersgr",
                         "C-ESIMST-0": "ErwerbStatus",
                         "C-IMST-0": "GeimpftGenesen",
                         "F-DATA": "Anz"}, inplace=True)


## Std-Analysis for Dataset 1

In [ ]:
dataset1.sample(5)

,PolBez,Altersgr,GebLand,Geschl,GeimpftGenesen,Anz
5720,BEZIMST-206,ALTGRIMST-9,GLIMST-1,C11-2,IMST-2,317
37113,BEZIMST-707,ALT10IMST-5,GLIMST-1,C11-2,IMST-1,820
34475,BEZIMST-701,ALT10IMST-3,GLIMST-1,C11-1,IMST-4,796
20683,BEZIMST-406,ALT10IMST-7,GLIMST-1,C11-2,IMST-4,730
25394,BEZIMST-417,ALT10IMST-3,GLIMST-1,C11-1,IMST-3,1839


In [ ]:
dataset1.head(5)

,PolBez,Altersgr,GebLand,Geschl,GeimpftGenesen,Anz
0,BEZIMST-101,ALTGRIMST-1,GLIMST-1,C11-1,IMST-1,1
1,BEZIMST-101,ALTGRIMST-1,GLIMST-1,C11-1,IMST-3,80
2,BEZIMST-101,ALTGRIMST-1,GLIMST-1,C11-1,IMST-4,216
3,BEZIMST-101,ALTGRIMST-1,GLIMST-1,C11-2,IMST-1,2
4,BEZIMST-101,ALTGRIMST-1,GLIMST-1,C11-2,IMST-3,86


In [ ]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53961 entries, 0 to 53960
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PolBez          53961 non-null  object
 1   Altersgr        53961 non-null  object
 2   GebLand         53961 non-null  object
 3   Geschl          53961 non-null  object
 4   GeimpftGenesen  53961 non-null  object
 5   Anz             53961 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 2.5+ MB


In [ ]:
dataset1.describe()

,Anz
count,53961.000000
mean,642.049703
std,2171.148389
min,1.000000
25%,44.000000
50%,169.000000
75%,508.000000
max,70338.000000


## Std-Analysis for Dataset 2

In [ ]:
dataset2.sample(5)

,Bundesland,Bildung,Altersgr,ErwerbStatus,GeimpftGenesen,Anz
1065,B00-7,BILIMST-99,ALT10IMST-4,ESIMST-2,IMST-3,10
1339,B00-9,BILIMST-3,ALT10IMST-7,ESIMST-2,IMST-4,2758
357,B00-3,BILIMST-2,ALT10IMST-6,ESIMST-2,IMST-1,7563
1240,B00-8,BILIMST-99,ALT10IMST-7,ESIMST-1,IMST-3,2
1261,B00-9,BILIMST-1,ALT10IMST-6,ESIMST-1,IMST-2,6507


In [ ]:
dataset2.head(5)

,Bundesland,Bildung,Altersgr,ErwerbStatus,GeimpftGenesen,Anz
0,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-1,IMST-1,611
1,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-1,IMST-2,285
2,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-1,IMST-3,396
3,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-1,IMST-4,472
4,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-2,IMST-1,471


In [ ]:
dataset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404 entries, 0 to 1403
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Bundesland      1404 non-null   object
 1   Bildung         1404 non-null   object
 2   Altersgr        1404 non-null   object
 3   ErwerbStatus    1404 non-null   object
 4   GeimpftGenesen  1404 non-null   object
 5   Anz             1404 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 65.9+ KB


In [ ]:
dataset2.describe()

,Anz
count,1404.000000
mean,3414.423789
std,5746.310880
min,1.000000
25%,310.500000
50%,1375.500000
75%,3926.000000
max,55581.000000


# Data Cleaning
In this section, the input data will be cleaned. For this manner, any wrong, missing, or irrelevant informations will be treated accordingly.

In [ ]:
geimpftGenesenDict = {"IMST-4": 0, #nothing
                      "IMST-1": 1, #vacc
                      "IMST-3": 2, #recov
                      "IMST-2": 3} #vacc + recov

dataset1["GebLand"] = dataset1["GebLand"].map({"GLIMST-1": "AT",
                                               "GLIMST-2": "AUSLAND"})
dataset1["Geschl"] = dataset1["Geschl"].map({"C11-1": "m",
                                             "C11-2": "f"})
dataset1["GeimpftGenesen"] = dataset1["GeimpftGenesen"].map(geimpftGenesenDict)

dataset2["GeimpftGenesen"] = dataset2["GeimpftGenesen"].map(geimpftGenesenDict)

,Bundesland,Bildung,Altersgr,ErwerbStatus,GeimpftGenesen,Anz
0,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-1,1,611
1,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-1,3,285
2,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-1,2,396
3,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-1,0,472
4,B00-1,BILIMST-1,ALT10IMST-4,ESIMST-2,1,471
...,...,...,...,...,...,...
1399,B00-9,BILIMST-99,ALT10IMST-7,ESIMST-1,0,14
1400,B00-9,BILIMST-99,ALT10IMST-7,ESIMST-2,1,31
1401,B00-9,BILIMST-99,ALT10IMST-7,ESIMST-2,3,1
1402,B00-9,BILIMST-99,ALT10IMST-7,ESIMST-2,2,4


## Duplicate Data

In [ ]:
dataset1["Anz"].sum()

34645644

In the above code we can see that the sum of records is approximately 4 times the total population of Austria. This strongly suggests that there may be duplicate data. And exactly that is the case: In ``PolBez`` there are records for each province and also for each federal country. This duplicate is not necessary, since one province can easily be assigned to its federal country (e.g. ``BEZIMST-304`` (= Wiener Neustadt) must be in ``B00-3`` (=Niederoesterreich)). So the records for the federal countries can be removed.

In [ ]:
dataset1 = dataset1.loc[dataset1["PolBez"].map(lambda val: "BEZIMST" in val)] #only provinces have the prefix BEZIMST
dataset1["Anz"].sum()

17322822

The above code still returns an amount which is twice as big as Austria's population. Similairly, the column ``Altersgr`` has duplicate records for different age groups. We can remove the non-specific records.

In [ ]:
dataset1 = dataset1.loc[dataset1["Altersgr"].map(lambda val: "ALTGRIMST" in val)] #only groups with prefix ALTGRIMST
dataset1["Anz"].sum()

8661411

Now we have removed all duplicate values. Let's continue cleaning our data.
## Null Values

In [ ]:
dataset1.isnull().sum()

PolBez            0
Altersgr          0
GebLand           0
Geschl            0
GeimpftGenesen    0
Anz               0
dtype: int64

In [ ]:
dataset2.isnull().sum()

Bundesland        0
Bildung           0
Altersgr          0
ErwerbStatus      0
GeimpftGenesen    0
Anz               0
dtype: int64

Since there are no null values, we can continue. (Es gibt null values in Bildung -- gekennzeichnet mit ``BILIMST-99`` siehe unten, KA wie behandeln (°~°))

In [ ]:
dataset2["Bildung"].value_counts()

BILIMST-1     288
BILIMST-2     288
BILIMST-3     288
BILIMST-4     288
BILIMST-99    252
Name: Bildung, dtype: int64

After this we are done cleaning our data and can save it into two datasets. ``korr1.csv`` for ``dataset1.csv`` and ``korr2.csv`` for ``dataset2.csv``

In [ ]:
dataset1.to_csv("/content/drive/MyDrive/output/korr1.csv", sep=";")
dataset2.to_csv("/content/drive/MyDrive/output/korr2.csv", sep=";")

# Data Preparation
## Numerical Values

In [ ]:
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32287 entries, 0 to 49823
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PolBez          32287 non-null  object
 1   Altersgr        32287 non-null  object
 2   GebLand         32287 non-null  object
 3   Geschl          32287 non-null  object
 4   GeimpftGenesen  32287 non-null  int64 
 5   Anz             32287 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.7+ MB


In [ ]:
dataset1.apply(pd.to_numeric, errors='ignore')

,PolBez,Altersgr,GebLand,Geschl,GeimpftGenesen,Anz
0,BEZIMST-101,ALTGRIMST-1,AT,m,1,1
1,BEZIMST-101,ALTGRIMST-1,AT,m,2,80
2,BEZIMST-101,ALTGRIMST-1,AT,m,0,216
3,BEZIMST-101,ALTGRIMST-1,AT,f,1,2
4,BEZIMST-101,ALTGRIMST-1,AT,f,2,86
...,...,...,...,...,...,...
49819,BEZIMST-923,ALTGRIMST-18,AUSLAND,m,0,18
49820,BEZIMST-923,ALTGRIMST-18,AUSLAND,f,1,162
49821,BEZIMST-923,ALTGRIMST-18,AUSLAND,f,3,42
49822,BEZIMST-923,ALTGRIMST-18,AUSLAND,f,2,7
